In [1]:
import io
import torch
import torch.onnx
import onnx
from unet import UNet
import onnxruntime
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
# from utils.dataset import BasicDataset

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
onnxpath = 'test_simpleclass.onnx'
print("torch.__version__:", torch.__version__)
print("onnx.__version__:", onnx.__version__)
print("onnxruntime.__version__:", onnxruntime.__version__)

torch.__version__: 1.4.0
onnx.__version__: 1.6.0
onnxruntime.__version__: 1.9.0


In [3]:
onnx_model = onnx.load(onnxpath)
check = onnx.checker.check_model(onnx_model)
print("check:", check)

check: None


In [4]:
input = torch.ones(1,1,32,32)
print(type(onnx_model))

<class 'onnx.onnx_ml_pb2.ModelProto'>


In [5]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension(批大小维度)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
net.load_state_dict(torch.load('net_params.pth'))
net.eval()

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (net): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [15]:
ort_session = onnxruntime.InferenceSession(onnxpath)
ort_x = torch.ones(1, 1, 32, 32, requires_grad=True)
print(ort_x)

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(ort_x)}
ort_outs = ort_session.run(None, ort_inputs)

torch_out = net(ort_x)
print(torch_out)
print(ort_outs)
# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]], requires_grad=True)
tensor([[ 0.1250,  0.2573, -0.3091, -0.2858,  0.1090,  0.3707, -0.1113, -0.1605,
          0.5539,  0.5565]], grad_fn=<AddmmBackward>)
[array([[ 0.12500986,  0.25731438, -0.30913767, -0.2857569 ,  0.10895298,
         0.37068182, -0.11133402, -0.16050792,  0.5538918 ,  0.5565266 ]],
      dtype=float32)]
Exported model has been tested with ONNXRuntime, and the result looks good!
